In [1]:
import scipy.io as sio
from scipy.stats import mode
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

%config InlineBackend.figure_format = 'retina'

In [2]:
BEST_PARAM_TABLE=np.load('best_params_4.npy')
ACCU_TABLE=np.load('accuracies_4.npy')

In [3]:
NUM_MODELS = ACCU_TABLE.shape[0]
NUM_DATASETS = ACCU_TABLE.shape[1]
NUM_PARTITIONS = ACCU_TABLE.shape[2]
NUM_TRIALS = ACCU_TABLE.shape[3]

In [4]:
AVG_TEST_ACCU_TABLE = np.zeros((NUM_MODELS,
                          NUM_DATASETS,
                          NUM_PARTITIONS))
AVG_TRAIN_ACCU_TABLE = np.zeros((NUM_MODELS,
                          NUM_DATASETS,
                          NUM_PARTITIONS))
AVG_VAL_ACCU_TABLE = np.zeros((NUM_MODELS,
                          NUM_DATASETS,
                          NUM_PARTITIONS))

In [5]:
# averaging test accuracies over 3 trials in all experiments
for model in range(NUM_MODELS):
    for dataset in range(NUM_DATASETS):
        for partition in range(NUM_PARTITIONS):
            AVG_TRAIN_ACCU_TABLE[model,dataset,partition] = \
                (ACCU_TABLE[model,dataset,partition,0][0]
                +ACCU_TABLE[model,dataset,partition,1][0]
                +ACCU_TABLE[model,dataset,partition,2][0])/3

In [6]:
# averaging test accuracies over 3 trials in all experiments
for model in range(NUM_MODELS):
    for dataset in range(NUM_DATASETS):
        for partition in range(NUM_PARTITIONS):
            AVG_VAL_ACCU_TABLE[model,dataset,partition] = \
                (ACCU_TABLE[model,dataset,partition,0][1]
                +ACCU_TABLE[model,dataset,partition,1][1]
                +ACCU_TABLE[model,dataset,partition,2][1])/3

In [7]:
# averaging test accuracies over 3 trials in all experiments
for model in range(NUM_MODELS):
    for dataset in range(NUM_DATASETS):
        for partition in range(NUM_PARTITIONS):
            AVG_TEST_ACCU_TABLE[model,dataset,partition] = \
                (ACCU_TABLE[model,dataset,partition,0][2]
                +ACCU_TABLE[model,dataset,partition,1][2]
                +ACCU_TABLE[model,dataset,partition,2][2])/3

In [8]:
AVG_TRAIN_ACCU_TABLE

array([[[0.75450006, 0.75566675, 0.76366697],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       [[0.85404075, 0.86026586, 0.86999793],
        [0.75466624, 0.75346449, 0.76800227],
        [0.73954115, 0.74146637, 0.7598338 ]],

       [[0.78879224, 0.79233318, 0.79466748],
        [0.7510414 , 0.75146585, 0.76783618],
        [0.73945771, 0.74253287, 0.76116646]],

       [[1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]]])

In [9]:
AVG_VAL_ACCU_TABLE

array([[[0.75441667, 0.75546667, 0.76233333],
        [0.73675   , 0.71106667, 0.67066667],
        [0.93208333, 0.91586667, 0.857     ]],

       [[0.84191667, 0.84066667, 0.82166667],
        [0.7485    , 0.7472    , 0.73533333],
        [0.73508333, 0.738     , 0.75333333]],

       [[0.78883333, 0.79106667, 0.79366667],
        [0.748     , 0.74533333, 0.73733333],
        [0.73483333, 0.73853333, 0.74733333]],

       [[0.84466667, 0.84853333, 0.83933333],
        [0.80658333, 0.78466667, 0.75966667],
        [0.92558333, 0.90986667, 0.86233333]]])

In [10]:
AVG_TEST_ACCU_TABLE

array([[[0.75733333, 0.75546667, 0.75441667],
        [0.78333333, 0.73373333, 0.68466667],
        [0.95133333, 0.93653333, 0.88516667]],

       [[0.856     , 0.84186667, 0.83575   ],
        [0.73866667, 0.73946667, 0.73491667],
        [0.73666667, 0.73666667, 0.73233333]],

       [[0.79566667, 0.79093333, 0.78775   ],
        [0.73666667, 0.73666667, 0.73633333],
        [0.74166667, 0.73986667, 0.73175   ]],

       [[0.862     , 0.84693333, 0.83758333],
        [0.811     , 0.80026667, 0.77116667],
        [0.93633333, 0.92      , 0.88491667]]])

In [12]:
RANKING_TABLE = AVG_TEST_ACCU_TABLE

In [13]:
def get_rank_list(arr):
    seq = sorted(arr)
    return [seq.index(v) for v in arr]

In [15]:
for dataset in range(NUM_DATASETS):
    for part in range(NUM_PARTITIONS):
        scores = []
        
        for model in range(NUM_MODELS):
            scores.append(AVG_TEST_ACCU_TABLE[model][dataset][part])
        
        rank_list = get_rank_list(scores)
        
        for model in range(NUM_MODELS):
            model_rank = rank_list[model]
            RANKING_TABLE[model][dataset][part] = model_rank

In [16]:
RANKING_TABLE

array([[[0., 0., 0.],
        [2., 0., 0.],
        [3., 3., 3.]],

       [[2., 2., 2.],
        [1., 2., 1.],
        [0., 0., 1.]],

       [[1., 1., 1.],
        [0., 1., 2.],
        [1., 1., 0.]],

       [[3., 3., 3.],
        [3., 3., 3.],
        [2., 2., 2.]]])